In [ ]:
import os
import random
from dotenv import load_dotenv
import nest_asyncio
import asyncio

load_dotenv()
nest_asyncio.apply()

api_key = os.getenv("OPENROUTER_API_KEY")
base_url = os.getenv("OPENROUTER_API_BASE_URL")
model_name = os.getenv("OPENROUTER_MODEL")

from agno.agent import Agent
from agno.models.openrouter import OpenRouter

model = OpenRouter(
    id=model_name,
    api_key=api_key,
    base_url=base_url,
    temperature=0.4
)

base_instruction = """
Ты участвуешь в рабочем чате.
Твоя задача — отвечать по делу, основываясь на своей роли.
Читай предыдущие сообщения и отвечай на них.
Пиши коротко, как в мессенджере.
"""

pm_instruction = """
Ты — Product Manager. Хочешь выпустить фичу как можно скорее.
Дави на сроки и говори о пользе для бизнеса.
"""

dev_instruction = """
Ты — Tech Lead. Ты за стабильность системы.
Критикуй быстрые решения и говори о рисках.
"""

qa_instruction = """
Ты — QA Engineer. Ты ищешь баги и проблемы.
Говори, что нельзя релизить с багами.
"""

cto_instruction = """
Ты — CTO. Ты решаешь споры.
Слушай всех и принимай взвешенные решения.
"""

topic = """
Обсуждаем фичу "Оплата криптой". 
Юристы говорят, что это серая зона, но конкуренты уже сделали.
Решаем: запускаем или ждем?
"""

pm = Agent(
    name="Ваня PM",
    model=model,
    instructions=[base_instruction, pm_instruction]
)

dev = Agent(
    name="Петя Разраб",
    model=model,
    instructions=[base_instruction, dev_instruction]
)

qa = Agent(
    name="Саша QA",
    model=model,
    instructions=[base_instruction, qa_instruction]
)

cto = Agent(
    name="Миша CTO",
    model=model,
    instructions=[base_instruction, cto_instruction]
)

agents = [pm, dev, qa, cto]

async def get_response(agent, history):
    response = await agent.arun(history, stream=False)
    content = response.content if response.content else ""
    return f"{agent.name}: {content}"

history = [{"role": "user", "content": topic}]

for i in range(10):
    agent = random.choice(agents)
    response = await get_response(agent, history)
    print(response)
    history.append({"role": "user", "content": response})
    await asyncio.sleep(15)